# Accelerating End-to-End Data Science Workflows # 

## 05 - KNN ##

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้ใช้การระบุตำแหน่งเพื่อนบ้านที่ใกล้ที่สุดแบบเร่งความเร็วด้วย GPU (GPU-accelerated k-nearest neighbors) เพื่อระบุจุดเชื่อมต่อถนนที่ใกล้ที่สุดกับโรงพยาบาล สมุดบันทึกนี้ครอบคลุมส่วนด้านล่าง:
1. [สภาพแวดล้อม (Environment)](#Environment)
2. [โหลดข้อมูล(Load-Data)](#Load-Data)
    * [จุดเชื่อมต่อถนน(Road-Nodes)](#Road-Nodes)
    * [โรงพยาบาล (Hospitals)](#Hospitals)
3. [เพื่อนบ้านที่ใกล้ที่สุด (K-Nearest Neighbors)](#K-Nearest-Neighbors)
    * [จุดเชื่อมต่อถนนที่ใกล้โรงพยาบาลแต่ละแห่งที่สุด](#Road-Nodes-Closest-to-Each-Hospital)
    * [การดูโรงพยาบาลที่เฉพาะเจาะจง](#Viewing-a-Specific-Hospital)


## สภาพเเวดล้อม ##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cudf
import cuml

## โหลดข้อมูล

### Road Nodes ###
เราจะเริ่มต้นด้วยการอ่านข้อมูล **จุดเชื่อมถนน (road nodes)** ของเรา

In [ ]:
# road_nodes = cudf.read_csv('./data/road_nodes_2-06.csv', dtype=['str', 'float32', 'float32', 'str'])
road_nodes = cudf.read_csv('./data/road_nodes.csv', dtype=['str', 'float32', 'float32', 'str'])

In [ ]:
road_nodes.dtypes

In [ ]:
road_nodes.shape

In [ ]:
road_nodes.head()

### โรงพยาบาล ###
ถัดไป เราจะโหลดข้อมูลโรงพยาบาล

In [ ]:
hospitals = cudf.read_csv('./data/clean_hospitals_full.csv')

In [ ]:
hospitals.dtypes

In [ ]:
hospitals.shape

In [ ]:
hospitals.head()

## K-Nearest Neighbors ##
เราจะใช้อัลกอริทึม **[เพื่อนบ้านใกล้ที่สุด k ตัว (k-nearest neighbors)](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)** เพื่อค้นหา **โหนดถนนที่ใกล้ที่สุด k โหนด** สำหรับโรงพยาบาลทุกแห่ง เราจะต้องทำการ Fit โมเดล KNN ด้วยข้อมูลถนน และจากนั้นจึงให้ตำแหน่งโรงพยาบาลแก่โมเดลที่เราฝึกมาแล้ว เพื่อให้มันส่งคืนถนนที่ใกล้ที่สุด

สร้างโมเดล k-nearest neighbors ชื่อ `knn` โดยใช้ Constructor `cuml.NearestNeighbors` และส่ง Argument ที่มีชื่อว่า `n_neighbors` โดยตั้งค่าเป็น `3`

สร้าง DataFrame ใหม่ชื่อ `road_locs` โดยใช้คอลัมน์ `east` และ `north` จาก `road_nodes` ลำดับของคอลัมน์ไม่สำคัญ ยกเว้นว่าเราจะต้องรักษามันให้สอดคล้องกันตลอดการทำงานหลายครั้ง ดังนั้นโปรดใช้ลำดับ `['east', 'north']`

ทำการ Fit โมเดล `knn` ด้วย `road_locs` โดยใช้เมธอด `knn.fit`

In [ ]:

knn = cuml.NearestNeighbors(n_neighbors=3)


In [ ]:

road_locs = road_nodes[['east', 'north']]
knn.fit(road_locs)


### จุดเชื่อมถนนที่ใกล้โรงพยาบาลแต่ละแห่งมากที่สุด ###
ใช้เมธอด `knn.kneighbors` เพื่อค้นหา **จุดเชื่อมถนน (road nodes)** ที่ใกล้ที่สุด 3 จุดสำหรับโรงพยาบาลแต่ละแห่ง เมธอด `knn.kneighbors` ต้องการ 2 อาร์กิวเมนต์: `X` ซึ่งคุณควรใช้คอลัมน์ `easting` และ `northing` ของ `hospitals` (อย่าลืมรักษำลำดับคอลัมน์ให้เหมือนตอนที่คุณ fit โมเดล `knn` ข้างต้น) และ `n_neighbors` ซึ่งคือจำนวนเพื่อนบ้านที่ต้องการค้นหา ในกรณีนี้คือ 3

`knn.kneighbors` จะคืนค่าเป็น `cudf dataframes` 2 ชุด ซึ่งคุณควรตั้งชื่อว่า `distances` และ `indices` ตามลำดับ

In [ ]:
distances, indices = knn.kneighbors(hospitals[['easting', 'northing']], 3) # order has to match the knn fit order (east, north)


### การดูข้อมูลโรงพยาบาลเฉพาะแห่ง ###
ตอนนี้เราสามารถใช้ `indices`, `hospitals` และ `road_nodes` เพื่อดึงข้อมูลเฉพาะของโรงพยาบาลที่ต้องการได้ ที่นี่เราจะมาดูข้อมูลโรงพยาบาลที่ **index 10** กัน ก่อนอื่นเราจะดูพิกัดกริดของโรงพยาบาลนี้:

In [ ]:
SELECTED_RESULT = 10
print('hospital coordinates:\n', hospitals.loc[SELECTED_RESULT, ['easting', 'northing']], sep='')

Now we view the road node IDs for the 3 closest road nodes:

In [ ]:
nearest_road_nodes = indices.iloc[SELECTED_RESULT, 0:3]
print('node_id:\n', nearest_road_nodes, sep='')

และสุดท้าย พิกัดกริดสำหรับจุดเชื่อมถนน (road nodes) ที่ใกล้ที่สุด 3 จุด ซึ่งเราสามารถยืนยันได้ว่าเรียงลำดับตามระยะทางที่เพิ่มขึ้นจากโรงพยาบาล:

In [ ]:
print('road_node coordinates:\n', road_nodes.loc[nearest_road_nodes, ['east', 'north']], sep='')

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**ทำได้ดีมาก!** ไปยัง [สมุดบันทึกถัดไป](3-06_xgboost.ipynb) กันเลย